
🏛️ ASSISTANT JURIDIQUE TUNISIE - SOLUTION COMPLÈTE

✅ Interface Gradio professionnelle

✅ Extraction PDF automatique

✅ LLM gratuit (Groq/Llama)

✅ Scraping web


In [ ]:


# ============================================================================
# INSTALLATION
# ============================================================================
print("🔧 Installation des packages...")
!pip install -q gradio groq chromadb sentence-transformers pypdf beautifulsoup4 requests lxml
print("✅ Installation terminée\n")

# ============================================================================
# IMPORTS
# ============================================================================
import os
import json
from pypdf import PdfReader
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import chromadb
import gradio as gr
from groq import Groq

# ============================================================================
# CONFIGURATION - CLÉ API GROQ GRATUITE
# ============================================================================
# Créez votre clé gratuite sur: https://console.groq.com/
os.environ['GROQ_API_KEY'] = 'secret_key_GROQ'

client = Groq(api_key=os.environ['GROQ_API_KEY'])

print("✅ Configuration OK\n")

# ============================================================================
# EXTRACTION DU PDF
# ============================================================================
print("📄 Extraction du PDF uploadé...")

pdf_path = "achat-et-la-vente-des-biens-immobiliers-par-les-etrangers-en-Tunisie-1.pdf"

def extraire_pdf(chemin_pdf):
    """Extrait le texte du PDF"""
    reader = PdfReader(chemin_pdf)
    texte_complet = ""

    for page in reader.pages:
        texte_complet += page.extract_text() + "\n\n"

    return texte_complet

texte_pdf = extraire_pdf(pdf_path)
print(f"✅ PDF extrait: {len(texte_pdf)} caractères\n")

# ============================================================================
# SCRAPING DU SITE WEB
# ============================================================================
print("🌐 Scraping du site web...")

def scraper_site(url):
    """Scrape le contenu du site"""
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraire le texte des paragraphes
        paragraphes = soup.find_all(['p', 'h1', 'h2', 'h3', 'li'])
        texte = '\n'.join([p.get_text().strip() for p in paragraphes if p.get_text().strip()])

        return texte
    except Exception as e:
        print(f"⚠️ Erreur scraping: {e}")
        return ""

url_site = "https://www.cyriljarnias.com/immobilier-international/investir-immobilier-tunisie-opportunites-regles-rendements/lois-regulations-immobilieres-tunisie/"
texte_web = scraper_site(url_site)

print(f"✅ Site scrapé: {len(texte_web)} caractères\n")

# ============================================================================
# DONNÉES DE BASE + PDF + WEB
# ============================================================================
print("📚 Compilation de toutes les sources...")

# Données de base
donnees_base = [
    {
        "titre": "Propriété immobilière",
        "contenu": "La propriété est le droit de jouir et disposer des choses de manière absolue selon le Code des Droits Réels tunisien de 1965.",
        "source": "Code des Droits Réels (1965)"
    },
    {
        "titre": "Droits d'enregistrement",
        "contenu": "Les droits d'enregistrement sont de 5% du prix de vente pour les propriétés bâties et terrains. Paiement dans les 30 jours.",
        "source": "Code Droits Enregistrement (1993)"
    },
    {
        "titre": "Permis de bâtir",
        "contenu": "Toute construction nécessite un permis de bâtir de la municipalité. Délai: 2 mois. Validité: 2 ans.",
        "source": "Code Aménagement Territoire (1994)"
    }
]

# Diviser le PDF en sections
sections_pdf = texte_pdf.split('\n\n')
donnees_pdf = [
    {
        "titre": f"Section PDF {i+1}",
        "contenu": section.strip(),
        "source": "PDF - Achat et vente par étrangers"
    }
    for i, section in enumerate(sections_pdf[:30])  # Limiter à 30 sections
    if len(section.strip()) > 100
]

# Diviser le contenu web en sections
sections_web = texte_web.split('\n')
donnees_web = [
    {
        "titre": f"Info Web {i+1}",
        "contenu": section.strip(),
        "source": "Site CyrilJarnias.com"
    }
    for i, section in enumerate(sections_web[:20])  # Limiter à 20 sections
    if len(section.strip()) > 100
]

# Combiner toutes les données
toutes_donnees = donnees_base + donnees_pdf + donnees_web

print(f"✅ Total: {len(toutes_donnees)} documents compilés\n")

# ============================================================================
# BASE VECTORIELLE
# ============================================================================
print("🗄️  Création de la base vectorielle...")

# Modèle d'embeddings
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# ChromaDB
chroma_client = chromadb.Client()

# Supprimer l'ancienne collection et créer une nouvelle
try:
    chroma_client.delete_collection(name="lois_tunisie_complete")
except:
    pass

collection = chroma_client.create_collection(name="lois_tunisie_complete")

# Indexer tous les documents
print("📥 Indexation des documents...")
for i, doc in enumerate(toutes_donnees):
    texte = f"{doc['titre']}: {doc['contenu']} (Source: {doc['source']})"
    collection.add(
        documents=[texte],
        metadatas=[{"titre": doc['titre'], "source": doc['source']}],
        ids=[f"doc_{i}"]
    )

    # Afficher la progression
    if (i + 1) % 10 == 0:
        print(f"  ✓ {i + 1}/{len(toutes_donnees)} documents indexés")

print(f"✅ {len(toutes_donnees)} documents indexés avec succès\n")

# ============================================================================
# SYSTÈME RAG AVEC GROQ (GRATUIT)
# ============================================================================
def repondre_question(question, historique):
    """
    Répond à une question en utilisant RAG + Groq (Llama gratuit)
    """
    # Recherche dans la base
    resultats = collection.query(
        query_texts=[question],
        n_results=5
    )

    # Contexte
    contexte = "\n\n".join(resultats['documents'][0])

    # Construire l'historique pour le contexte
    messages = []

    # Ajouter le contexte système
    system_prompt = f"""Tu es un assistant juridique tunisien expert en droit immobilier.
Tu dois répondre UNIQUEMENT en te basant sur le contexte fourni ci-dessous.

CONTEXTE JURIDIQUE:
{contexte}

RÈGLES:
- Réponds en français de manière professionnelle
- Cite toujours la source (loi, code, article)
- Si l'info n'est pas dans le contexte: dis "Information non disponible dans ma base"
- Sois précis et concis
- Structure tes réponses clairement"""

    messages.append({"role": "system", "content": system_prompt})

    # Ajouter l'historique
    for humain, assistant in historique:
        if humain:
            messages.append({"role": "user", "content": humain})
        if assistant:
            messages.append({"role": "assistant", "content": assistant})

    # Ajouter la question actuelle
    messages.append({"role": "user", "content": question})

    # Appeler Groq (gratuit!)
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",  # Modèle gratuit et puissant
            messages=messages,
            temperature=0.1,
            max_tokens=800
        )

        reponse = response.choices[0].message.content

        # Ajouter les sources
        sources = "\n\n📚 **Sources consultées:**\n"
        for i, meta in enumerate(resultats['metadatas'][0][:3], 1):
            sources += f"- {meta['source']}\n"

        return reponse + sources

    except Exception as e:
        return f"❌ Erreur: {e}"

# ============================================================================
# INTERFACE GRADIO
# ============================================================================
print("🎨 Création de l'interface Gradio...\n")

# CSS personnalisé
css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
}
.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 20px;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:

    # Header
    gr.HTML("""
    <div class="header">
        <h1>🏛️ Assistant Juridique Immobilier - Tunisie</h1>
        <p>Expert en droit immobilier tunisien | Base enrichie avec lois, PDF et sources web</p>
    </div>
    """)

    # Description
    gr.Markdown("""
    ### 💬 Posez vos questions sur :
    - 🏠 Achat et vente immobilière
    - 👥 Transactions par des étrangers
    - 📜 Droits d'enregistrement
    - 🏗️ Permis de bâtir
    - 🔐 Hypothèques et garanties
    - 👨‍⚖️ Copropriété et succession
    """)

    # Interface de chat
    chatbot = gr.Chatbot(
        height=500,
        label="💬 Conversation",
        avatar_images=("👤", "🤖")
    )

    # Zone de texte
    with gr.Row():
        msg = gr.Textbox(
            placeholder="Posez votre question juridique...",
            label="Votre question",
            scale=4
        )
        send = gr.Button("📤 Envoyer", variant="primary", scale=1)

    # Exemples
    gr.Examples(
        examples=[
            "Quels sont les droits d'enregistrement en Tunisie?",
            "Un étranger peut-il acheter une maison en Tunisie?",
            "Comment obtenir un permis de bâtir?",
            "Quelles sont les règles pour les terres agricoles?",
            "Quelle est la procédure pour acheter un bien immobilier?",
        ],
        inputs=msg,
        label="💡 Questions d'exemple"
    )

    # Stats
    gr.Markdown(f"""
    ---
    📊 **Base de connaissances:** {len(toutes_donnees)} documents juridiques

    📚 **Sources:** Code des Droits Réels, PDF juridique, Site web spécialisé

    🤖 **Modèle:** Llama 3.3 70B (via Groq - gratuit)
    """)

    # Logique du chat
    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message = repondre_question(user_message, history[:-1])
        history[-1][1] = bot_message
        return history

    # Events
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    send.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

    # Clear
    clear = gr.Button("🗑️ Nouvelle conversation")
    clear.click(lambda: None, None, chatbot, queue=False)

# ============================================================================
# LANCEMENT
# ============================================================================
print("="*70)
print("✅ SYSTÈME PRÊT!")
print("="*70)
print("\n🚀 Lancement de l'interface Gradio...\n")

demo.launch(share=True, debug=True)

🔧 Installation des packages...
✅ Installation terminée

✅ Configuration OK

📄 Extraction du PDF uploadé...
✅ PDF extrait: 9701 caractères

🌐 Scraping du site web...
✅ Site scrapé: 40917 caractères

📚 Compilation de toutes les sources...
✅ Total: 9 documents compilés

🗄️  Création de la base vectorielle...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


📥 Indexation des documents...
✅ 9 documents indexés avec succès

🎨 Création de l'interface Gradio...



/tmp/ipython-input-3423974863.py:251: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3423974863.py:251: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3423974863.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3423974863.py:273: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from Fa

✅ SYSTÈME PRÊT!

🚀 Lancement de l'interface Gradio...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://95b7fc71869628c4e4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://95b7fc71869628c4e4.gradio.live
